In [1]:
import os
import pandas as pd 
import numpy as np
import pyomo.environ as pe
import pyomo.opt as po
import yaml
from powerutils.models import *
from powerutils.models.shortterm import StochasticShortTermMinShedObjective

from util import (max_budget,
                  update_budget_constraint,
                  cut_first_stage_solution)

currentdir=os.getcwd()
DATA_DIR = os.path.join(currentdir, 'data','input')
RESULTS_DIR = os.path.join(currentdir, 'data','output')

In [2]:
def update_objective(solver, instance, weight):
    instance.weight = weight
    StochasticShortTermMinShedObjective.obj_min_expected_shed(instance)
    solver.set_objective(instance.obj_min_expected_shed)

In [3]:
# Name your folder
pftype = 'lpdc'
case = 'Harvey25-B50-blue140000'

In [4]:
# overhead
specs_dir = os.path.join(DATA_DIR, 'specs')

#reduced grid
specs_fs_filename = os.path.join(specs_dir, 'specs-harvey-sandbag-disc.yaml')
specs_ss_filename = os.path.join(specs_dir, 'specs-lpdc-reduced-ACTIVSg2000.yaml')

# outputs
res_dir = os.path.join(RESULTS_DIR, case, 'sp')
os.makedirs(res_dir, exist_ok=True)

In [5]:
solver = po.SolverFactory('gurobi_persistent')
solver.options['timelimit'] = 3 * 60 * 60
solver.options['mipgap'] = 0.00

In [6]:
specs = dict()

with open(specs_fs_filename) as fh:
    specs.update(yaml.full_load(fh))

with open(specs_ss_filename) as fh:
    specs.update(yaml.full_load(fh))

if pftype == 'lpdc':
    model = StochasticShortTermMinShedLPDC()
elif pftype == 'lpac':
    model = StochasticShortTermMinShedLPAC()
else:
    msg = f'Power flow type \'{pftype}\' not recognized.'
    raise ValueError(msg)

In [7]:
def con_gen_dispatch(model, n, g, omega):
    if specs['I_g'][g]:
        return model.zeta[n, g, omega] == model.alpha[n, omega]
    else:
        return model.zeta[n, g, omega] == 0

In [8]:
import time
start = time.time()

instance = pe.ConcreteModel()
model.setup(instance, **specs)
instance.con_gen_dispatch_eq = pe.Constraint(instance.NxG, instance.Omega, rule=con_gen_dispatch)
solver.set_instance(instance)

print('model setup completed in ',time.time()-start,' seconds')

    unordered data source (type: set).  This WILL potentially lead to
    nondeterministic behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-30
model setup completed in  103.52994775772095  seconds


In [9]:
# Varying Weights (lambda)
w_min= 140000
w_max= 150000
w_inc= 10000
set_w = np.arange(w_min, w_max + w_inc, w_inc)
#set_w = [140000,150000]

f = 50
update_budget_constraint(solver, instance, f)

for f in set_w:
    print(f)
    start = time.time()
    update_objective(solver, instance, f)
    logfile = os.path.join(res_dir, f'{pftype}-f{f}.log')
    zipfile = os.path.join(res_dir, f'{pftype}-f{f}.zip')
    result = solver.solve(logfile=logfile, tee=True)
    status = str(result['Solver'][0].get('Termination condition').value)
    if not status == 'infeasible':
        solution = Solution.from_solved_instance(instance)
        solution.to_zip(zipfile, variables=['x', 'objective','z'])
    else:
        print('Instance with f={} has no feasible solution!'.format(f))
    print('optimization completed in ',time.time()-start,' seconds')

140000
Set parameter TimeLimit to value 10800
Set parameter MIPGap to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 443279 rows, 172553 columns and 1128793 nonzeros
Model fingerprint: 0xbef3ed39
Variable types: 87875 continuous, 84678 integer (84678 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+04]
  Objective range  [7e-01, 2e+04]
  Bounds range     [1e+00, 3e+00]
  RHS range        [6e-01, 9e+03]
Presolve removed 411897 rows and 138893 columns
Presolve time: 3.21s
Presolved: 31382 rows, 33660 columns, 138196 nonzeros
Variable types: 33400 continuous, 260 integer (260 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   20175    5.0386606e+07   5.141196e+03   3.875672e+09      5s
   25531    4.9503495e+07   2.990959e+03   2

   885   352 1679944.28   23   17 1697730.68 1638172.65  3.51%   451 1194s
   946   368 1662897.94   15   24 1697730.68 1638789.27  3.47%   442 1233s
   986   390 1691591.95   20   15 1697730.68 1640524.82  3.37%   436 1254s
  1044   405 1689763.75   26   12 1697730.68 1641503.32  3.31%   425 1282s
  1099   419 1690282.61   23   13 1697730.68 1642465.80  3.26%   420 1306s
  1165   451 1695928.98   25   14 1697730.68 1643577.47  3.19%   411 1329s
  1224   456 1677524.42   24   16 1697730.68 1644476.57  3.14%   404 1355s
  1285   478 1648634.19   18   26 1697730.68 1646152.94  3.04%   400 1387s
  1341   497 1689571.05   13   27 1697730.68 1646332.20  3.03%   393 1462s
  1439   505     cutoff   21      1697730.68 1647824.97  2.94%   400 1501s
  1490   529 1652295.57   18   21 1697730.68 1648123.62  2.92%   398 1535s
  1581   536     cutoff   14      1697730.68 1649120.79  2.86%   388 1574s
  1663   560 1690075.80   12   34 1697730.68 1649960.07  2.81%   382 1612s
  1750   565 1674790.13  

    32    29 1798835.91   10   51 1804702.20 1676982.09  7.08%  1894  454s
    37    31     cutoff   11      1804702.20 1676982.09  7.08%  1735  460s
    42    29     cutoff   10      1804702.20 1679852.69  6.92%  1576  473s
    50    35 1727661.89    4   65 1804702.20 1679852.69  6.92%  1639  485s
    59    41 1729973.90    5   65 1804702.20 1679852.69  6.92%  1475  496s
    65    49 1748349.48    6   64 1804702.20 1679852.69  6.92%  1476  503s
    73    61 1752419.60    7   51 1804702.20 1679852.69  6.92%  1353  509s
    86    68 1794680.10    9   45 1804702.20 1679852.69  6.92%  1189  513s
    95    67 1802950.60   10   39 1804702.20 1679852.69  6.92%  1107  528s
   104    73     cutoff   11      1804702.20 1679852.69  6.92%  1082  533s
   116    81     cutoff    9      1804702.20 1679852.69  6.92%  1008  540s
   126    89 1798020.43    9   41 1804702.20 1679852.69  6.92%   966  545s
   138    89 1802310.05   10   39 1804702.20 1679852.69  6.92%   905  555s
   146    91     cutoff  

  4744   942     cutoff   31      1804702.20 1770657.49  1.89%   477 3807s
  4755   940 1770657.49   20   46 1804702.20 1770657.49  1.89%   482 3840s
  4766   934 1778141.43   21   45 1804702.20 1770657.49  1.89%   487 3869s
  4786   941 1798355.80   23   50 1804702.20 1770657.49  1.89%   491 3891s
  4814   923     cutoff   28      1804702.20 1770657.49  1.89%   493 3924s
  4847   911 1790613.85   24   31 1804702.20 1770657.49  1.89%   497 3949s
  4885   884     cutoff   36      1804702.20 1770657.49  1.89%   499 3980s
  4923   854     cutoff   34      1804702.20 1773534.96  1.73%   503 4001s
  4970   834     cutoff   31      1804702.20 1777888.82  1.49%   503 4028s
  5002   802     cutoff   22      1804702.20 1778504.29  1.45%   506 4047s
  5047   771     cutoff   25      1804702.20 1782992.87  1.20%   504 4072s
  5072   759     cutoff   26      1804702.20 1791453.90  0.73%   505 4077s

Cutting planes:
  Gomory: 6
  Implied bound: 1084
  Projected implied bound: 53
  Clique: 11
  MIR: